# Pizza Routing
[This is a companion script for a pgRouting tutorial.]()

In [1]:
import psycopg2
import csv

## Change user='' to your database username

In [2]:
conn = psycopg2.connect("dbname=routing user=ariel")
cur = conn.cursor()

## Find the "source" way. 
This will be the road we route to all the pizza nodes from. To find the 'source' go to https://www.openstreetmap.org/ and find the way you'd like to route from. In my case I used the road in front of Penn Station, https://www.openstreetmap.org/way/195743190.

In [3]:
way_osm_id = 195743190
cur.execute("SELECT source FROM ways WHERE osm_id = %s", (way_osm_id,))
source = cur.fetchone()[0]
print("My source: " + str(source))

My source: 181766


## Get all the pizza nodes.

In [4]:
cur.execute("SELECT * FROM osm_nodes WHERE tag_value = 'pizza'")
osm_ids = []
for record in cur:
    osm_ids.append(record[0])
    
print("We're going to route to: " + str(len(osm_ids)) + " pizza nodes.")

We're going to route to: 300 pizza nodes.


## Find the closest streets to each pizza node.

In [5]:
osm_ids_streets = []
for osm_id in osm_ids:
    cur.execute("SELECT source\
        FROM ways\
        ORDER BY ways.the_geom <-> (SELECT the_geom FROM osm_nodes WHERE osm_id = %s limit 10) limit 1;", (osm_id,))
    osm_ids_streets.append(cur.fetchone()[0])

print("We've got: " + str(len(osm_ids_streets)) + " nearest streets. Should be the same number as pizza nodes.")

We've got: 300 nearest streets. Should be the same number as pizza nodes.


## This function will output each route to a csv file, which we can then load into QGIS.

In [6]:
def writeRoute(route):
    print("writing route of length: " + str(len(route)))
    with open('routes.csv', 'a') as f:
        writer = csv.writer(f, lineterminator='\n')
        for r in route:
            writer.writerow(r)

## Finally, loop through all the nearest streets to each pizza node and route to them from a common source. 
Tip: add a *break* to the loop and test just one route first!

In [7]:
for osm_id in osm_ids_streets:
    cur.execute("select ST_AsText(the_geom), agg_cost from (SELECT * FROM pgr_dijkstra('\
        SELECT gid as id, source, target, length as cost\
        FROM ways',\
        %s, %s, false)) as x, ways where x.edge = ways.gid;", (source, osm_id,));
    writeRoute(cur.fetchall())

writing route of length: 79
writing route of length: 94
writing route of length: 160
writing route of length: 252
